In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. Setup paths and device
local_model_path = "../models"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Load Model and Tokenizer
print(f"Loading model from {local_model_path}...")
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(local_model_path)

# Move model to GPU if available for speed
model.to(device)

# 3. Your Input Text
article_text = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Locally nicknamed "La Dame de Fer" (French for "The Iron Lady"), it was constructed from 1887 to 1889
as the centerpiece of the 1889 World's Fair. It has become a global cultural icon of France
and one of the most recognizable structures in the world. The tower is 330 metres (1,083 ft) tall,
about the same height as an 81-storey building, and the tallest structure in Paris.
"""

# 4. Tokenize (Pre-processing)
# We must truncate long texts because BART has a limit (usually 1024 tokens)
inputs = tokenizer(
    article_text,
    max_length=1024,   # BART's limit
    truncation=True,   # Cut off text if it's too long
    return_tensors="pt" # Return PyTorch tensors
)

# Move inputs to the same device as the model
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

# 5. Generate Summary (Inference)
# This is where the magic happens. We don't just run model(), we run model.generate()
summary_ids = model.generate(
    input_ids,
    attention_mask=attention_mask,
    num_beams=4,       # Beam search makes the summary higher quality
    min_length=30,     # Force it to be at least this long
    max_length=100,    # Cap the summary length
    early_stopping=True
)

# 6. Decode (Post-processing)
# Convert the output numbers back into words
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("--- Original ---")
print(article_text)
print("\n--- Summary ---")
print(summary_text)

Loading model from ../models...
--- Original ---

The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Locally nicknamed "La Dame de Fer" (French for "The Iron Lady"), it was constructed from 1887 to 1889 
as the centerpiece of the 1889 World's Fair. It has become a global cultural icon of France 
and one of the most recognizable structures in the world. The tower is 330 metres (1,083 ft) tall, 
about the same height as an 81-storey building, and the tallest structure in Paris. 


--- Summary ---
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It was constructed from 1887 to 1889 as the centerpiece of the 1889 World's Fair. The tower is 330 metres (1,083 ft) tall, about the same height as an 81-storey building.


In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSeq2SeqLM, AutoTokenizer

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Falconsai/text_summarization",
    dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(
    "Falconsai/text_summarization"
)
input_text = """Plants are remarkable organisms that produce their own food using a method called photosynthesis.
This process involves converting sunlight, carbon dioxide, and water into glucose, which provides energy for growth.
Plants play a crucial role in sustaining life on Earth by generating oxygen and serving as the foundation of most ecosystems."""
input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)

output = model.generate(**input_ids, cache_implementation="static")
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. Setup the device (Check for GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_name = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Move the Model to the device
# We chain .to(device) immediately after loading
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

ARTICLE = """
Hugging Face: Revolutionizing Natural Language Processing
Introduction
In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.
The Birth of Hugging Face
Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name "Hugging Face" was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology.
Transformative Innovations
Hugging Face is best known for its open-source contributions, particularly the "Transformers" library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis.
Key Contributions:
1. **Transformers Library:** The Transformers library provides a unified interface for more than 50 pre-trained models, simplifying the development of NLP applications. It allows users to fine-tune these models for specific tasks, making it accessible to a wider audience.
2. **Model Hub:** Hugging Face's Model Hub is a treasure trove of pre-trained models, making it simple for anyone to access, experiment with, and fine-tune models. Researchers and developers around the world can collaborate and share their models through this platform.
3. **Hugging Face Transformers Community:** Hugging Face has fostered a vibrant online community where developers, researchers, and AI enthusiasts can share their knowledge, code, and insights. This collaborative spirit has accelerated the growth of NLP.
Democratizing AI
Hugging Face's most significant impact has been the democratization of AI and NLP. Their commitment to open-source development has made powerful AI models accessible to individuals, startups, and established organizations. This approach contrasts with the traditional proprietary AI model market, which often limits access to those with substantial resources.
By providing open-source models and tools, Hugging Face has empowered a diverse array of users to innovate and create their own NLP applications. This shift has fostered inclusivity, allowing a broader range of voices to contribute to AI research and development.
Industry Adoption
The success and impact of Hugging Face are evident in its widespread adoption. Numerous companies and institutions, from startups to tech giants, leverage Hugging Face's technology for their AI applications. This includes industries as varied as healthcare, finance, and entertainment, showcasing the versatility of NLP and Hugging Face's contributions.
Future Directions
Hugging Face's journey is far from over. As of my last knowledge update in September 2021, the company was actively pursuing research into ethical AI, bias reduction in models, and more. Given their track record of innovation and commitment to the AI community, it is likely that they will continue to lead in ethical AI development and promote responsible use of NLP technologies.
Conclusion
Hugging Face's story is one of transformation, collaboration, and empowerment. Their open-source contributions have reshaped the NLP landscape and democratized access to AI. As they continue to push the boundaries of AI research, we can expect Hugging Face to remain at the forefront of innovation, contributing to a more inclusive and ethical AI future. Their journey reminds us that the power of open-source collaboration can lead to groundbreaking advancements in technology and bring AI within the reach of many.
"""

inputs = tokenizer(ARTICLE, return_tensors="pt", truncation=True)

# 3. Move the Data (Inputs) to the device
# We move the input_ids tensor to the GPU before passing it to the model
input_ids = inputs["input_ids"].to(device)

summary_ids = model.generate(
    input_ids,
    max_length=1000,
    min_length=30,
    do_sample=False
)

# Decoding is usually done on CPU, so we don't need to move anything back manually
# The tokenizer handles the device transfer internally during decoding if necessary
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print([{'summary_text': summary_text}])

Using device: cuda
[{'summary_text': 'and AI as a whole Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry. This article will explore the story and significance of Hugging Face Hugging Face. The Birth of Hugging Face Hugging Face Hugging Face is best known for its open-source contributions, particularly the "Transformers Library:** The Transformers library provides'}]
